In [89]:
# import all necessary packages
import pandas as pd

In [90]:
# define the path of the dataset
mnist_path = "/usr/scratch/badile31/msc22f11/msc22f11/PlayGround/mnist_train.csv"
# read it in as a pandas dataframe
mnist_train_df = pd.read_csv(mnist_path)
mnist_train_df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
# define how many rows to read in
n_rows = 20
# read in the first n_rows rows of the dataframe
mnist_train_n_rows_df = mnist_train_df[:n_rows]
mnist_train_n_rows_df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
# separate the dataframe into the features and the labels
mnist_train_n_rows_features_df = mnist_train_n_rows_df.drop(["label"], axis=1)
mnist_train_n_rows_labels_df = mnist_train_n_rows_df["label"]

In [93]:
# scale the feature values to be between 0 and 1
mnist_train_n_rows_features_scaled_df = mnist_train_n_rows_features_df.div(255)
mnist_train_n_rows_features_scaled_df.head()

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
# special handling for FP8 type where we set up the conversion first
import pathlib
import ctypes

from ctypes import c_uint8, c_double, c_float
from ctypes import byref, Structure


class flexfloat_desc_t(Structure):
    _fields_ = [("exp_bits", c_uint8), ("frac_bits", c_uint8)]


class flexfloat_t(Structure):
    _fields_ = [("value", c_double), ("desc", flexfloat_desc_t)]


fp64_desc = flexfloat_desc_t(11, 52)
fp32_desc = flexfloat_desc_t(8, 23)
fp16_desc = flexfloat_desc_t(5, 11)
fp16alt_desc = flexfloat_desc_t(8, 7)
fp8_desc = flexfloat_desc_t(5, 2)
fp8alt_desc = flexfloat_desc_t(4, 3)

lib_path = "/usr/scratch/badile31/msc22f11/msc22f11/PlayGround/flexfloat/src/libflexfloat.so"
ff_lib = ctypes.CDLL(lib_path)

ff_get_float = ff_lib.ff_get_float
ff_get_float.restype = c_float


class ff:
    def __init__(self, value: float, desc: flexfloat_desc_t = fp64_desc):
        self.desc = desc
        self.value = value
        self.a = flexfloat_t(value, desc)
        ff_lib.ff_init_float(byref(self.a), c_float(value), desc)

    def __add__(self, b):
        ff_res = flexfloat_t(0.0, self.desc)
        ff_lib.ff_add(byref(ff_res), byref(self.a), byref(b.a))
        return 0

In [95]:
# function to convert float32 to binary representation
import struct

def float32_to_bin(value):
    return ''.join(f'{c:0>8b}' for c in struct.pack('!f', value))

In [96]:
"""
We have to handle denormalized numbers:
    
    +INF will be represented in FP8 as 0 11111 00 
    -INF will be represented in FP8 as 1 11111 00
    +0 will be represented in FP8 as 0 00000 00
    -0 will be represented in FP8 as 1 00000 00
    NaN will be represented in FP8 as X 11111 MM (at least one of the MMM bits is set, sign bit is don't care)

According to https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8556098 denormalized transprecision numbers
will be represented by their high precision counterparts. In these cases we have to make sure that we do not adjust
the exponent. In the other cases we adjust the exponent and cut the mantissa.
"""

from numpy import binary_repr

# this function returns an 8 character string representing the binary representation of the FP8 number
def float32_to_fp8(value):
    max_exp_fp32 = int('11111111', 2)
    min_exp_fp32 = int('00000000', 2)
    exp_bias_fp32 = 2 ** (8 - 1) - 1
    exp_bias_fp8 = 2 ** (5 - 1) - 1
    # get the binary representation of the number
    binstr = float32_to_bin(value)
    # extract sign, exponent and mantissa bits
    sign = binstr[0]
    exponent = binstr[1:9]
    mantissa = binstr[9:]
    # check if the number is denormalized
    # we start by checking if all exponent bits are asserted
    if(int(exponent) == max_exp_fp32):
        # if so, we check if the mantissa is all zeros (will result in +/-INF)
        if(int(mantissa) == 0):
            return '0b' + sign + exponent[:5] + mantissa[:2]
        # if not, we have to return a NaN
        else:
            return '0b' + sign + exponent[:5] + '01'
    # if both exponent and mantissa are zero we will return +/-0
    elif (int(exponent) == min_exp_fp32 and int(mantissa) == 0): 
        return '0b' + sign + exponent[:5] + mantissa[:2]
    else :
        # if not, we adjust the exponent and cut the mantissa
        exponent_fp8 = binary_repr(int(exponent, 2) - exp_bias_fp32 + exp_bias_fp8, width=5)
        mantissa_fp8 = mantissa[:2]
        return '0b' + sign + exponent_fp8 + mantissa_fp8


In [97]:
# exponent is stored in two's complement
def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val  

In [98]:
def convert_to_fp8_decimal(binstr):

    # extract sign, exponent and mantissa bits
    sign = binstr[0]
    num_sign_bits = 1
    # print(f'Sign:     ({num_sign_bits} bit)  = {sign}')
    exponent = binstr[1:6]
    num_exp_bits = len(exponent)
    # print(f'Exponent: ({num_exp_bits} bit)  = {exponent}')
    mantissa = binstr[6:]
    num_mant_bits = len(mantissa)
    # print(f'Mantissa: ({num_mant_bits} bit) = {mantissa}')

    exp_bias_fp8 = 2 ** (5 - 1) - 1
    dec_val_fp8 = (-1)**(int(sign, 2)) * (1 + (int(mantissa, 2))/(2**num_mant_bits)) * 2**(twos_comp(int(exponent, 2), num_exp_bits) - exp_bias_fp8)
    if(int(sign, 2) == 0 and int(exponent, 2) == 0 and int(mantissa, 2) == 0):
        dec_val_fp8 = 0
    # print("\nBinary to floating point number (FP8) conversion using formula: ", dec_val_fp8)
    return dec_val_fp8
    

In [99]:
# define the special FP8 values
fp8_nans = ['01111101', '01111110', '01111111', '11111101', '11111110', '11111111']
# print fp8_nans as integers
for nan in fp8_nans:
    print(f' NaN: {nan} = {int(nan, 2)}')
fp8_pinf = '01111100'
print(f'+INF: {fp8_pinf} = {int(fp8_pinf, 2)}')
fp8_ninf = '11111100'
print(f'-INF: {fp8_ninf} = {int(fp8_ninf, 2)}')

 NaN: 01111101 = 125
 NaN: 01111110 = 126
 NaN: 01111111 = 127
 NaN: 11111101 = 253
 NaN: 11111110 = 254
 NaN: 11111111 = 255
+INF: 01111100 = 124
-INF: 11111100 = 252


In [100]:
fp8_int_values = range(0, 256)
# remove the special FP8 values
fp8_int_values = [x for x in fp8_int_values if x not in [int(nan, 2) for nan in fp8_nans] + [int(fp8_pinf, 2), int(fp8_ninf, 2)]]

In [101]:
import random
def clear_special_values(data_df):
    nan_cnt = 0
    inf_cnt = 0
    for col in data_df.columns:
        for row in data_df.index:
            if(data_df[col][row] in fp8_nans):
                nan_cnt += 1
                print(f'NaN in {col} at row {row}: {data_df[col][row]}')
                # set value to random integer in range 0-255 that is not NaN or INF
                data_df[col][row] = random.choice(fp8_int_values)
            elif(data_df[col][row] == fp8_pinf):
                inf_cnt += 1
                print(f'+INF in {col} at row {row}: {data_df[col][row]}')
            elif(data_df[col][row] == fp8_ninf):
                inf_cnt += 1
                print(f'-INF in {col} at row {row}: {data_df[col][row]}')
    print(f'NaN count: {nan_cnt}')
    print(f'INF count: {inf_cnt}')

    return data_df

In [102]:
# convert to target data type
PREC = 16
if PREC == 64:
    mnist_train_n_rows_features_scaled_df = mnist_train_n_rows_features_scaled_df.astype("float64")
elif PREC == 32:
    mnist_train_n_rows_features_scaled_df = mnist_train_n_rows_features_scaled_df.astype("float32")
elif PREC == 16:
    mnist_train_n_rows_features_scaled_df = mnist_train_n_rows_features_scaled_df.astype("float16")
elif PREC == 8:
    mnist_train_n_rows_features_scaled_df = mnist_train_n_rows_features_scaled_df.astype("float32")
    # convert to FP8
    mnist_train_n_rows_features_scaled_df = mnist_train_n_rows_features_scaled_df.applymap(float32_to_fp8)
    # clear special values
    mnist_train_n_rows_features_scaled_df = clear_special_values(mnist_train_n_rows_features_scaled_df)

In [103]:
type(mnist_train_n_rows_features_scaled_df.iloc[0][0])

numpy.float16

In [104]:
mnist_train_n_rows_features_scaled_df.iloc[0][0]

0.0

In [105]:
data_type = type(mnist_train_n_rows_features_scaled_df.iloc[0][0])
if data_type == str:
    feature_type = 'float8'
else:
    feature_type = str(type(mnist_train_n_rows_features_scaled_df.iloc[0][0])).split(".")[-1].split("'")[0]
print("Data type of the features: {}".format(feature_type))
# get the data type of the labels
label_type = str(type(int(mnist_train_n_rows_labels_df.iloc[0]))).split("<")[-1].split(">")[0].split("class")[-1].replace("'", "").strip()
print("Data type of the labels: {}".format(label_type))

Data type of the features: float16
Data type of the labels: int


In [106]:
# write the feature data into a binary file
for row in mnist_train_n_rows_features_scaled_df.values:
    bin_row = bytearray(row)
    f = open(str(n_rows) + '_features_' + str(feature_type), 'ab')
    f.write(bin_row)
    f.close()
print("Wrote {} rows of features to binary file: {}".format(n_rows, str(n_rows) + '_features_' + str(feature_type)))

Wrote 20 rows of features to binary file: 20_features_float16


In [88]:
# write the label data into a binary file
for row in mnist_train_n_rows_labels_df.values:
    bin_row = int(row).to_bytes(8, byteorder='little')
    f = open(str(n_rows) + '_labels_' + str(label_type), 'ab')
    f.write(bin_row)
    f.close()
print("Wrote {} rows of labels to binary file: {}".format(n_rows, str(n_rows) + '_labels_' + str(label_type)))

Wrote 20 rows of labels to binary file: 20_labels_int
